In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras

In [4]:
df = pd.read_csv('Dataset_raw_20230414-20230630.csv')

In [5]:
df.head()

,time,Building Power/Total Active Power,HVAC Power/Total Active Power,Breakroom VAV1_02/Current Cooling Setpoint,Breakroom VAV1_02/Current Heating Setpoint,Collab Lounge VAV1_01/Current Cooling Setpoint,Collab Lounge VAV1_01/Current Heating Setpoint,Conference VAV1_04/Current Cooling Setpoint,Conference VAV1_04/Current Heating Setpoint,GE Additive Manufacturing VAV2_01/Current Cooling Setpoint,...,Offices 108_110 VAV1_06/Current Heating Setpoint,Open Office VAV1_05/Current Cooling Setpoint,Open Office VAV1_05/Current Heating Setpoint,Training Room Nine VAV1_08/Current Cooling Setpoint,Training Room Nine VAV1_08/Current Heating Setpoint,Training Room Twelve VAV1_09/Current Cooling Setpoint,Training Room Twelve VAV1_09/Current Heating Setpoint,Weather Station/Outside Humidity,Weather Station/Outside Temp,Weather Station/Solar Radiation
0,2023-04-14 00:00:18.671,6410.6180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-04-14 00:00:53.673,6533.5146,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-04-14 00:01:29.676,6425.5580,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-04-14 00:01:52.684,6545.1597,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-04-14 00:03:16.693,7480.0654,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df.fillna(method='ffill', inplace=True)
df.dropna(how='any', axis=0, inplace=True)
df = df.drop('Building Power/Total Active Power', axis=1)

# convert the dates to datetime to be able to acess it
sr = pd.to_datetime(df['time'])
df['DayofWeek'] = sr.dt.weekday
df['DayofYear'] = sr.dt.dayofyear

In [7]:
df.head()

,time,HVAC Power/Total Active Power,Breakroom VAV1_02/Current Cooling Setpoint,Breakroom VAV1_02/Current Heating Setpoint,Collab Lounge VAV1_01/Current Cooling Setpoint,Collab Lounge VAV1_01/Current Heating Setpoint,Conference VAV1_04/Current Cooling Setpoint,Conference VAV1_04/Current Heating Setpoint,GE Additive Manufacturing VAV2_01/Current Cooling Setpoint,GE Additive Manufacturing VAV2_01/Current Heating Setpoint,...,Open Office VAV1_05/Current Heating Setpoint,Training Room Nine VAV1_08/Current Cooling Setpoint,Training Room Nine VAV1_08/Current Heating Setpoint,Training Room Twelve VAV1_09/Current Cooling Setpoint,Training Room Twelve VAV1_09/Current Heating Setpoint,Weather Station/Outside Humidity,Weather Station/Outside Temp,Weather Station/Solar Radiation,DayofWeek,DayofYear
8382,2023-04-14 08:35:44.056,15497.989,74.894394,72.894394,74.212616,72.212616,73.106316,71.106316,77.0,75.0,...,72.73064,72.43602,70.43602,72.47923,70.47923,55.0,62.1,91.0,4,104
8383,2023-04-14 08:35:44.224,15342.122,74.894394,72.894394,74.212616,72.212616,73.106316,71.106316,77.0,75.0,...,72.73064,72.43602,70.43602,72.47923,70.47923,55.0,62.1,91.0,4,104
8384,2023-04-14 08:35:45.223,14988.585,74.894394,72.894394,74.212616,72.212616,73.106316,71.106316,77.0,75.0,...,72.73064,72.43602,70.43602,72.47923,70.47923,55.0,62.1,91.0,4,104
8385,2023-04-14 08:35:45.956,14988.585,74.894394,72.894394,74.212616,72.212616,73.106316,71.106316,77.0,75.0,...,72.73064,72.43602,70.43602,72.47923,70.47923,55.0,62.1,91.0,4,104
8386,2023-04-14 08:35:46.224,15189.883,74.894394,72.894394,74.212616,72.212616,73.106316,71.106316,77.0,75.0,...,72.73064,72.43602,70.43602,72.47923,70.47923,55.0,62.1,91.0,4,104


In [8]:
# randomly get 10% of the data for the model
dataset = df.sample(frac=0.1)

In [9]:
dataset.shape

(277903, 37)

In [10]:
dataset

,time,HVAC Power/Total Active Power,Breakroom VAV1_02/Current Cooling Setpoint,Breakroom VAV1_02/Current Heating Setpoint,Collab Lounge VAV1_01/Current Cooling Setpoint,Collab Lounge VAV1_01/Current Heating Setpoint,Conference VAV1_04/Current Cooling Setpoint,Conference VAV1_04/Current Heating Setpoint,GE Additive Manufacturing VAV2_01/Current Cooling Setpoint,GE Additive Manufacturing VAV2_01/Current Heating Setpoint,...,Open Office VAV1_05/Current Heating Setpoint,Training Room Nine VAV1_08/Current Cooling Setpoint,Training Room Nine VAV1_08/Current Heating Setpoint,Training Room Twelve VAV1_09/Current Cooling Setpoint,Training Room Twelve VAV1_09/Current Heating Setpoint,Weather Station/Outside Humidity,Weather Station/Outside Temp,Weather Station/Solar Radiation,DayofWeek,DayofYear
1847865,2023-06-06 12:17:42.854,53559.6500,74.901050,72.901050,74.400840,72.400840,71.783640,69.783640,77.0,75.0,...,72.740810,74.829790,72.829790,72.480370,70.480370,32.0,81.3,761.0,1,157
1223191,2023-05-16 08:25:36.092,9016.8680,74.898834,72.898834,74.222920,72.222920,71.788100,69.788100,77.0,75.0,...,72.747925,73.563370,71.563370,72.480370,70.480370,83.0,57.1,65.0,1,136
1216445,2023-05-16 04:17:16.933,1282.9424,80.000000,60.000000,80.000000,60.000000,80.000000,60.000000,80.0,60.0,...,60.000000,80.000000,60.000000,80.000000,60.000000,83.0,62.6,0.0,1,136
1157604,2023-05-14 03:33:00.086,1288.5457,80.000000,60.000000,80.000000,60.000000,80.000000,60.000000,80.0,60.0,...,60.000000,80.000000,60.000000,80.000000,60.000000,76.0,68.6,0.0,6,134
781836,2023-05-03 13:30:04.453,13716.3510,74.892180,72.892180,74.217200,72.217200,71.781400,69.781400,77.0,75.0,...,72.732666,73.546165,71.546165,72.470200,70.470200,55.0,55.0,1078.0,2,123
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2728483,2023-06-29 14:01:11.662,104236.9400,74.908806,72.908806,74.403114,72.403114,71.792560,69.792560,77.0,75.0,...,70.690094,74.844970,72.844970,73.783295,71.783295,56.0,86.2,853.0,3,180
2146942,2023-06-14 10:00:56.755,15286.3280,74.901050,72.901050,74.400840,72.400840,71.785866,69.785866,77.0,75.0,...,70.678894,74.834854,72.834854,72.482630,70.482630,70.0,67.6,626.0,2,165
2022443,2023-06-11 09:14:21.340,1291.4681,80.000000,60.000000,80.000000,60.000000,80.000000,60.000000,80.0,60.0,...,60.000000,80.000000,60.000000,80.000000,60.000000,74.0,63.5,18.0,6,162
2136245,2023-06-14 08:03:22.285,11218.3125,74.901050,72.901050,74.400840,72.400840,71.785866,69.785866,77.0,75.0,...,70.678894,74.834854,72.834854,72.482630,70.482630,85.0,62.6,302.0,2,165


In [11]:
# get rid of the time colume and convert the rest of the columes to float
dataset = dataset.drop('time', axis=1)
dataset = dataset.astype(float)

In [12]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [13]:
# the data using standard scaler 
scaler = preprocessing.StandardScaler()
scaler = scaler.fit(dataset)
scaled_dataset = scaler.fit_transform(dataset)

In [14]:
X = [] # test data
y = [] # prediction data

n_predict = 1 # amount of data that is predicted
n_past = 100 # amount of data used for the prediction

In [15]:
for i in range(n_past, len(dataset) - n_predict + 1):
    X.append(scaled_dataset[i - n_past:i, 0:dataset.shape[1]])
    y.append(scaled_dataset[i + n_predict - 1:i + n_predict, 0])

In [16]:
# turn X and y into arrays
X, y = np.array(X), np.array(y)

In [17]:
len(X), len(y)

(277803, 277803)

In [18]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM
from keras.optimizers.schedules import InverseTimeDecay
from keras.optimizers import Adam

In [19]:
# gradient clipping
initial_learning_rate = 0.001
decay_steps = 1000
decay_rate = 0.95
learning_rate_fn = InverseTimeDecay(initial_learning_rate, decay_steps, decay_rate)

# create optimizer
optimizer = Adam(learning_rate = learning_rate_fn, clipvalue = 0.5)

In [20]:
# define the model
model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(LSTM(32, activation='relu', return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1]))
model.compile(optimizer=optimizer, loss='mse')

In [21]:
X.shape

(277803, 100, 36)

In [22]:
y.shape

(277803, 1)

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100, 64)           25856     
                                                                 
 lstm_1 (LSTM)               (None, 32)                12416     
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 38305 (149.63 KB)
Trainable params: 38305 (149.63 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [24]:
#fit model
#history = model.fit(X, y, epochs=10, batch_size=10, validation_split=0.1, verbose=1)
model.fit(X, y, epochs=50, batch_size=10, validation_split=0.1, verbose=1)

Epoch 1/50
25003/25003 [==============================] - 1572s 62ms/step - loss: 1.0004 - val_loss: 1.0012
Epoch 2/50
17596/25003 [====================>.........] - ETA: 8:05 - loss: 0.9984

KeyboardInterrupt: 

In [ ]:
# plot how the training went
plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')